In [ ]:
def unique_preserve_order(seq):
    seen = set()
    result = []
    for item in seq:
        if item not in seen:
            seen.add(item)
            result.append(item)
    return result

import pandas as pd
import json

def unique_preserve_order(seq):
    seen = set()
    result = []
    for item in seq:
        if item not in seen:
            seen.add(item)
            result.append(item)
    return result

# --- Load your data used for training the model ---
ridership_df = pd.read_csv('/Users/fahmi.taib/Desktop/Deployment Code Test/very_new_finalised_ridership_route.csv', low_memory=False)
ridership_df['date'] = pd.to_datetime(ridership_df['date'], format='%m/%d/%y', errors='coerce')
ridership_df = ridership_df.dropna(subset=['date'])

ranked_routes = pd.read_csv('/Users/fahmi.taib/Desktop/Deployment Code Test/top_200_routes_by_ridership.csv')
top_routes = ranked_routes.head(180)['route_no'].astype(str)
ridership_df = ridership_df[ridership_df['route_no'].astype(str).isin(top_routes)]

agg_df = ridership_df.groupby(['route_no', 'date']).agg({
    'ridership_total': 'sum',
    'hour': 'max',
    'depot': 'first',
    'route_id': 'first'  # Keep route_id here to get its categories, even if not used in model features
}).reset_index()

depot_categories = [str(x) for x in ["29", "7", "10", "22", "27", "2", "5", "37", "4", "38"]]

route_no_categories = [str(x) for x in [
    "PJ01", "100", "200", "201", "300", "302", "303", "400", "401", "500", "600", "601", "T504", "T505",
    "T506", "T507", "T508", "T509", "T510", "T511", "T512", "T545", "301", "T201", "SJ01", "SA03", "SEWA1",
    "T715", "T753", "T754", "T756", "T774", "T776", "T778", "T780", "T781", "T782", "T786", "T787", "T788",
    "T789", "T790", "T791", "402", "751", "752", "754", "770", "771", "783", "SA08", "Subang HQ", "T783",
    "T850", "708", "750", "753", "782", "T562", "T785", "SA01", "SA02", "SU13A", "T300", "T301", "T302",
    "MAHA1", "T304", "T406", "T571", "T406B", "T602", "506", "540", "T250", "T450", "T580", "T604", "202",
    "250", "251", "253", "590", "602", "641", "650", "651", "652", "T581", "T601", "T605", "T640", "580",
    "581", "541", "T221", "151", "173", "180", "191", "220", "222", "780", "801", "802", "T202", "T203",
    "T203B", "T222", "254", "772", "821", "T251", "BET16", "PBD1", "T200", "190", "SEWA 2", "T603", "170",
    "822", "851", "T600", "171", "T582", "640", "T221B", "BET17", "P101", "P102", "P103", "P105", "P106",
    "P108", "T407", "T569", "T567", "T568", "MS01", "DS01", "DS01(PM)", "420", "450", "PAVILION BUKI",
    "AJ2A", "T783B", "AJ03", "T785B", "SEWA3", "PTPM", "GP03", "T566", "T559", "T543", "T582B", "T757B",
    "T774B", "T786B", "KJ03", "T542", "T757", "T778B", "421", "T224", "T223", "T350", "KLG2A", "T303",
    "T351", "SA04", "T120", "HLB1", "451", "SA06", "T173"
]]

route_id_categories = [str(x) for x in [
    "1219", "736", "215", "218", "219", "220", "222", "223", "224", "225", "663", "226", "227", "228", "1081",
    "1082", "1083", "1084", "1085", "1086", "1087", "1088", "1089", "1179", "221", "853", "531", "667", "1152",
    "566", "570", "571", "573", "651", "653", "655", "724", "725", "723", "730", "731", "732", "733", "729",
    "734", "497", "498", "500", "501", "502", "721", "940", "741", "726", "742", "495", "496", "499", "720",
    "1096", "728", "530", "666", "993", "1248", "1247", "1245", "1243", "1249", "1228", "1227", "1246", "1212",
    "1221", "548", "1013", "1222", "1223", "631", "1242", "480", "546", "1241", "547", "1274", "633", "1277",
    "956", "1279", "558", "1275", "616", "1276", "594", "858", "559", "673", "629", "877", "848", "849", "850",
    "478", "488", "679", "716", "717", "671", "672", "674", "615", "630", "718", "632", "486", "487", "613",
    "856", "1110", "873", "874", "876", "844", "846", "847", "835", "837", "838", "854", "855", "1213", "857",
    "851", "834", "880", "859", "860", "904", "852", "875", "1155", "845", "647", "678", "870", "878", "881",
    "882", "646", "871", "675", "714", "1206", "861", "1060", "1061", "1062", "1064", "1065", "1066", "755",
    "1182", "1101", "1102", "1161", "1002", "1004", "474", "481", "483", "990", "477", "790", "1208", "173",
    "792", "1210", "1250", "1178", "1187", "1100", "1030", "1091", "1207", "1126", "1214", "1209", "797",
    "1090", "656", "1205", "482", "866", "865", "549", "779", "606", "550", "668", "884", "1015", "1159", "923"
]]

# Remove duplicates while preserving order
depot_categories = unique_preserve_order(depot_categories)
route_no_categories = unique_preserve_order(route_no_categories)
route_id_categories = unique_preserve_order(route_id_categories)

# Save to Python file for import
with open('/Users/fahmi.taib/Desktop/Deployment Code Test/fixed_categories.py', 'w') as f:
    f.write("# Auto-generated fixed category lists for consistent encoding\n\n")
    f.write(f"depot_categories = {json.dumps(depot_categories, indent=4)}\n\n")
    f.write(f"route_no_categories = {json.dumps(route_no_categories, indent=4)}\n\n")
    f.write(f"route_id_categories = {json.dumps(route_id_categories, indent=4)}\n")

print("Fixed category lists saved to fixed_categories.py")


Fixed category lists saved to fixed_categories.py


In [9]:
from collections import Counter
import pandas as pd
from pandas.api.types import CategoricalDtype

def find_duplicates(seq):
    counts = Counter(seq)
    return [item for item, count in counts.items() if count > 1]

def unique_preserve_order_clean(seq):
    seen = set()
    result = []
    for item in seq:
        item_clean = str(item).strip().lower()
        if item_clean not in seen:
            seen.add(item_clean)
            result.append(item)
    return result

route_no_categories = [str(x) for x in [
    "402","PJ01", "100", "200", "201", "300", "302", "303", "400", "401", "500", "600", "601", "T504", "T505",
    "T506", "T507", "T508", "T509", "T510", "T511", "T512", "T545", "301", "T201", "SJ01", "SA03", "SEWA1",
    "T715", "T753", "T754", "T756", "T774", "T776", "T778", "T780", "T781", "T782", "T786", "T787", "T788",
    "T789", "T790", "T791", "402", "751", "752", "754", "770", "771", "783", "SA08", "Subang HQ", "T783",
    "T850", "708", "750", "753", "782", "T562", "T785", "SA01", "SA02", "SU13A", "T300", "T301", "T302",
    "MAHA1", "T304", "T406", "T571", "T406B", "T602", "506", "540", "T250", "T450", "T580", "T604", "202",
    "250", "251", "253", "590", "602", "641", "650", "651", "652", "T581", "T601", "T605", "T640", "580",
    "581", "541", "T221", "151", "173", "180", "191", "220", "222", "780", "801", "802", "T202", "T203",
    "T203B", "T222", "254", "772", "821", "T251", "BET16", "PBD1", "T200", "190", "SEWA 2", "T603", "170",
    "822", "851", "T600", "171", "T582", "640", "T221B", "BET17", "P101", "P102", "P103", "P105", "P106",
    "P108", "T407", "T569", "T567", "T568", "MS01", "DS01", "DS01(PM)", "420", "450", "PAVILION BUKI",
    "AJ2A", "T783B", "AJ03", "T785B", "SEWA3", "PTPM", "GP03", "T566", "T559", "T543", "T582B", "T757B",
    "T774B", "T786B", "KJ03", "T542", "T757", "T778B", "421", "T224", "T223", "T350", "KLG2A", "T303",
    "T351", "SA04", "T120", "HLB1", "451", "SA06", "T173"
]]

print(f"Original length: {len(route_no_categories)}")
print("Duplicates before cleaning:", find_duplicates(route_no_categories))

# Remove None/NaN
route_no_categories = [x for x in route_no_categories if pd.notna(x)]

# Deduplicate with cleaning
route_no_categories = unique_preserve_order_clean(route_no_categories)

print(f"Length after cleaning: {len(route_no_categories)}")
print("Duplicates after cleaning:", find_duplicates(route_no_categories))

# Now safe to create CategoricalDtype
route_no_cat_type = CategoricalDtype(categories=route_no_categories, ordered=True)


Original length: 176
Duplicates before cleaning: ['402']
Length after cleaning: 175
Duplicates after cleaning: []


In [26]:
import pandas as pd
import numpy as np
from xgboost import XGBRegressor
from sklearn.model_selection import train_test_split, RandomizedSearchCV
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from sklearn.preprocessing import StandardScaler
from scipy.stats import randint, uniform
import joblib

# --- Manually define fixed mappings as dictionaries ---
depot_mapping = {
    "29": 0,
    "7": 1,
    "10": 2,
    "22": 3,
    "27": 4,
    "2": 5,
    "5": 6,
    "37": 7,
    "4": 8,
    "38": 9,
}

route_no_mapping = {
    "PJ01": 0,
    "100": 1,
    "200": 2,
    "201": 3,
    "300": 4,
    "302": 5,
    "303": 6,
    "400": 7,
    "401": 8,
    "500": 9,
    "600": 10,
    "601": 11,
    "T504": 12,
    "T505": 13,
    "T506": 14,
    "T507": 15,
    "T508": 16,
    "T509": 17,
    "T510": 18,
    "T511": 19,
    "T512": 20,
    "T545": 21,
    "301": 22,
    "T201": 23,
    "SJ01": 24,
    "SA03": 25,
    "SEWA1": 26,
    "T715": 27,
    "T753": 28,
    "T754": 29,
    "T756": 30,
    "T774": 31,
    "T776": 32,
    "T778": 33,
    "T780": 34,
    "T781": 35,
    "T782": 36,
    "T786": 37,
    "T787": 38,
    "T788": 39,
    "T789": 40,
    "T790": 41,
    "T791": 42,
    "402": 43,
    "751": 44,
    "752": 45,
    "754": 46,
    "770": 47,
    "771": 48,
    "783": 49,
    "SA08": 50,
    "Subang HQ": 51,
    "T783": 52,
    "T850": 53,
    "708": 54,
    "750": 55,
    "753": 56,
    "782": 57,
    "T562": 58,
    "T785": 59,
    "SA01": 60,
    "SA02": 61,
    "SU13A": 62,
    "T300": 63,
    "T301": 64,
    "T302": 65,
    "MAHA1": 66,
    "T304": 67,
    "T406": 68,
    "T571": 69,
    "T406B": 70,
    "T602": 71,
    "506": 72,
    "540": 73,
    "T250": 74,
    "T450": 75,
    "T580": 76,
    "T604": 77,
    "202": 78,
    "250": 79,
    "251": 80,
    "253": 81,
    "590": 82,
    "602": 83,
    "641": 84,
    "650": 85,
    "651": 86,
    "652": 87,
    "T581": 88,
    "T601": 89,
    "T605": 90,
    "T640": 91,
    "580": 92,
    "581": 93,
    "541": 94,
    "T221": 95,
    "151": 96,
    "173": 97,
    "180": 98,
    "191": 99,
    "220": 100,
    "222": 101,
    "780": 102,
    "801": 103,
    "802": 104,
    "T202": 105,
    "T203": 106,
    "T203B": 107,
    "T222": 108,
    "254": 109,
    "772": 110,
    "821": 111,
    "T251": 112,
    "BET16": 113,
    "PBD1": 114,
    "T200": 115,
    "190": 116,
    "SEWA 2": 117,
    "T603": 118,
    "170": 119,
    "822": 120,
    "851": 121,
    "T600": 122,
    "171": 123,
    "T582": 124,
    "640": 125,
    "T221B": 126,
    "BET17": 127,
    "P101": 128,
    "P102": 129,
    "P103": 130,
    "P105": 131,
    "P106": 132,
    "P108": 133,
    "T407": 134,
    "T569": 135,
    "T567": 136,
    "T568": 137,
    "MS01": 138,
    "DS01": 139,
    "DS01(PM)": 140,
    "420": 141,
    "450": 142,
    "PAVILION BUKIT JALIL": 143,
    "AJ2A": 144,
    "T783B": 145,
    "AJ03": 146,
    "T785B": 147,
    "SEWA3": 148,
    "PTPM": 149,
    "GP03": 150,
    "T566": 151,
    "T559": 152,
    "T543": 153,
    "T582B": 154,
    "T757B": 155,
    "T774B": 156,
    "T786B": 157,
    "KJ03": 158,
    "T542": 159,
    "T757": 160,
    "T778B": 161,
    "421": 162,
    "T224": 163,
    "T223": 164,
    "T350": 165,
    "KLG2A": 166,
    "T303": 167,
    "T351": 168,
    "SA04": 169,
    "T120": 170,
    "HLB1": 171,
    "451": 172,
    "SA06": 173,
    "T173": 174
}

# --- Load ranked routes ---
ranked_routes = pd.read_csv('/Users/fahmi.taib/Desktop/Deployment Code Test/top_200_routes_by_ridership.csv')

# --- Load ridership data ---
ridership_df = pd.read_csv('/Users/fahmi.taib/Desktop/Deployment Code Test/very_new_finalised_ridership_route.csv', low_memory=False)
ridership_df['date'] = pd.to_datetime(ridership_df['date'], format='%m/%d/%y', errors='coerce')
ridership_df = ridership_df.dropna(subset=['date'])

# --- Load holiday data ---
holiday_df = pd.read_csv('/Users/fahmi.taib/Desktop/Deployment Code Test/Holiday 2024.csv')
holiday_df['StartDate'] = holiday_df['StartDate'].astype(str) + '-2024'
holiday_df['EndDate'] = holiday_df['EndDate'].astype(str) + '-2024'
holiday_df['StartDate'] = pd.to_datetime(holiday_df['StartDate'], format='%d-%b-%Y', errors='coerce')
holiday_df['EndDate'] = pd.to_datetime(holiday_df['EndDate'], format='%d-%b-%Y', errors='coerce')

all_holiday_dates = []
for _, row in holiday_df.iterrows():
    if pd.isna(row['StartDate']) or pd.isna(row['EndDate']):
        continue
    all_holiday_dates.extend(pd.date_range(start=row['StartDate'], end=row['EndDate']))

holiday_dates_set = set(all_holiday_dates)

# --- Filter to top 180 routes ---
top_routes = ranked_routes.head(180)['route_no'].astype(str)
ridership_df = ridership_df[ridership_df['route_no'].astype(str).isin(top_routes)]

# Flag holidays in ridership data
ridership_df['is_holiday'] = ridership_df['date'].apply(lambda x: 1 if x in holiday_dates_set else 0)

# --- Aggregate ridership per route per day ---
agg_df = ridership_df.groupby(['route_no', 'date']).agg({
    'ridership_total': 'sum',
    'hour': 'max',
    'depot': 'first',
    'is_holiday': 'first'
}).reset_index()

# --- Calculate hours left before schedule ends ---
END_HOUR = 24
agg_df['hours_left'] = END_HOUR - agg_df['hour']
agg_df['hours_left'] = agg_df['hours_left'].clip(lower=0)
# Calculate hours elapsed in the day (max hour in data)
agg_df['hours_elapsed'] = agg_df['hour']

# Calculate average ridership per hour so far
agg_df['avg_ridership_per_hour'] = agg_df['ridership_total'] / agg_df['hours_elapsed'].replace(0, 1)  # avoid division by zero

# Estimate total ridership for the full day by multiplying avg ridership per hour by 24 hours
agg_df['ridership_total'] = agg_df['avg_ridership_per_hour'] * 24

# Alternatively, estimate ridership remaining by multiplying avg ridership per hour by hours left
agg_df['estimated_ridership_remaining'] = agg_df['avg_ridership_per_hour'] * agg_df['hours_left']


# --- Add temporal features ---
agg_df['day_of_week'] = agg_df['date'].dt.dayofweek
agg_df['month'] = agg_df['date'].dt.month

# --- Map categorical columns using your fixed mappings ---
agg_df['depot_enc'] = agg_df['depot'].map(depot_mapping).fillna(-1).astype(int)
agg_df['route_no_enc'] = agg_df['route_no'].map(route_no_mapping).fillna(-1).astype(int)


# Save to CSV for further use or inspection
agg_df.to_csv('/Users/fahmi.taib/Desktop/Deployment Code Test/agg_ridership_with_estimates.csv', index=False)

print("Aggregated ridership with estimates saved successfully.")

# # --- Prepare features and target ---
# features = ['route_no_enc', 'day_of_week', 'month', 'depot_enc', 'is_holiday', 'hours_left']
# X = agg_df[features]
# y = agg_df['ridership_total']

# # --- Train/test split ---
# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# # --- Scale features ---
# scaler = StandardScaler()
# X_train_scaled = scaler.fit_transform(X_train)
# X_test_scaled = scaler.transform(X_test)

# # --- Define XGBoost model ---
# xgb = XGBRegressor(random_state=42, objective='reg:squarederror')

# # --- Hyperparameter distributions ---
# param_dist = {
#     'n_estimators': randint(100, 500),
#     'max_depth': randint(3, 10),
#     'learning_rate': uniform(0.01, 0.3),
#     'subsample': uniform(0.6, 0.4),
#     'colsample_bytree': uniform(0.6, 0.4),
#     'gamma': uniform(0, 5),
#     'reg_alpha': uniform(0, 1),
#     'reg_lambda': uniform(0, 1)
# }

# # --- Randomized Search ---
# random_search = RandomizedSearchCV(
#     estimator=xgb,
#     param_distributions=param_dist,
#     n_iter=50,
#     scoring='neg_mean_absolute_error',
#     cv=3,
#     verbose=1,
#     random_state=42,
#     n_jobs=-1
# )

# print("Starting hyperparameter tuning...")
# random_search.fit(X_train_scaled, y_train)

# print(f"Best parameters found: {random_search.best_params_}")

# # --- Predict and evaluate ---
# best_model = random_search.best_estimator_
# y_pred = best_model.predict(X_test_scaled)

# mae = mean_absolute_error(y_test, y_pred)
# mse = mean_squared_error(y_test, y_pred)
# rmse = np.sqrt(mse)
# r2 = r2_score(y_test, y_pred)

# print("\nXGBoost Model Evaluation:")
# print(f"MAE: {mae:.2f}")
# print(f"MSE: {mse:.2f}")
# print(f"RMSE: {rmse:.2f}")
# print(f"R-squared: {r2:.4f}")

# # --- Save model and scaler ---
# joblib.dump(best_model, '/Users/fahmi.taib/Desktop/Deployment Code Test/xgb_ridership_model.pkl')
# joblib.dump(scaler, '/Users/fahmi.taib/Desktop/Deployment Code Test/xgb_feature_scaler.pkl')

# print("Model and scaler saved successfully.")

# --- Calculate average ridership per hour and estimated total ridership ---
agg_df['hours_elapsed'] = agg_df['hour'].replace(0, 1)  # avoid division by zero
agg_df['avg_ridership_per_hour'] = agg_df['ridership_total'] / agg_df['hours_elapsed']
agg_df['ridership_total'] = agg_df['avg_ridership_per_hour'] * 24  # full day estimate

# --- Prepare features and target ---
features = ['route_no_enc', 'day_of_week', 'month', 'depot_enc', 'is_holiday', 'hours_left']
X = agg_df[features]
y = agg_df['ridership_total']  

# --- Train/test split ---
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# --- Scale features ---
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# --- Define XGBoost model ---
xgb = XGBRegressor(random_state=42, objective='reg:squarederror')

# --- Hyperparameter distributions ---
param_dist = {
    'n_estimators': randint(100, 500),
    'max_depth': randint(3, 10),
    'learning_rate': uniform(0.01, 0.3),
    'subsample': uniform(0.6, 0.4),
    'colsample_bytree': uniform(0.6, 0.4),
    'gamma': uniform(0, 5),
    'reg_alpha': uniform(0, 1),
    'reg_lambda': uniform(0, 1)
}

# --- Randomized Search ---
random_search = RandomizedSearchCV(
    estimator=xgb,
    param_distributions=param_dist,
    n_iter=50,
    scoring='neg_mean_absolute_error',
    cv=3,
    verbose=1,
    random_state=42,
    n_jobs=-1
)

print("Starting hyperparameter tuning...")
random_search.fit(X_train_scaled, y_train)

print(f"Best parameters found: {random_search.best_params_}")

# --- Predict and evaluate ---
best_model = random_search.best_estimator_
y_pred = best_model.predict(X_test_scaled)

mae = mean_absolute_error(y_test, y_pred)
mse = mean_squared_error(y_test, y_pred)
rmse = np.sqrt(mse)
r2 = r2_score(y_test, y_pred)

print("\nXGBoost Model Evaluation:")
print(f"MAE: {mae:.2f}")
print(f"MSE: {mse:.2f}")
print(f"RMSE: {rmse:.2f}")
print(f"R-squared: {r2:.4f}")

# --- Save model and scaler ---
joblib.dump(best_model, '/Users/fahmi.taib/Desktop/Deployment Code Test/xgb_ridership_model.pkl')
joblib.dump(scaler, '/Users/fahmi.taib/Desktop/Deployment Code Test/xgb_feature_scaler.pkl')

print("Model and scaler saved successfully.")



Aggregated ridership with estimates saved successfully.
Starting hyperparameter tuning...
Fitting 3 folds for each of 50 candidates, totalling 150 fits
Best parameters found: {'colsample_bytree': np.float64(0.6174415087017735), 'gamma': np.float64(4.972752553986705), 'learning_rate': np.float64(0.1509833541972829), 'max_depth': 8, 'n_estimators': 472, 'reg_alpha': np.float64(0.883494022266259), 'reg_lambda': np.float64(0.7477187738974139), 'subsample': np.float64(0.9812287388095813)}

XGBoost Model Evaluation:
MAE: 123.82
MSE: 288485.98
RMSE: 537.11
R-squared: 0.9093
Model and scaler saved successfully.


In [16]:
from pandas.api.types import CategoricalDtype

def unique_preserve_order(seq):
    seen = set()
    result = []
    for item in seq:
        if item not in seen:
            seen.add(item)
            result.append(item)
    return result

# Clean your categories before using them
route_no_categories = unique_preserve_order(route_no_categories)
depot_categories = unique_preserve_order(depot_categories)

# Now create CategoricalDtype objects
depot_cat_type = CategoricalDtype(categories=depot_categories, ordered=True)
route_no_cat_type = CategoricalDtype(categories=route_no_categories, ordered=True)

# Proceed with encoding
agg_df['depot_enc'] = agg_df['depot'].astype(depot_cat_type).cat.codes
agg_df['route_no_enc'] = agg_df['route_no'].astype(route_no_cat_type).cat.codes


In [17]:
print(f"Original length: {len(route_no_categories)}")
route_no_categories = unique_preserve_order(route_no_categories)
print(f"Cleaned length: {len(route_no_categories)}")


Original length: 175
Cleaned length: 175
